In [3]:
import os
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("ggplot")
%matplotlib inline

from tqdm import tqdm_notebook, tnrange
from itertools import chain
from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from sklearn.model_selection import train_test_split

import tensorflow as tf

from keras.models import Model, load_model
from keras.layers import Input, BatchNormalization, Activation, Dense, Dropout
from keras.layers.core import Lambda, RepeatVector, Reshape
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D, GlobalMaxPool2D
from keras.layers.merge import concatenate, add
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

Using TensorFlow backend.


In [4]:
ids = next(os.walk("data/images"))[2] # list of names all images in the given path
ids_m = next(os.walk("data/masks"))[2] # list of names all images in the given path

print("No. of images = ", len(ids))

No. of images =  5000


In [5]:
im_width = 32
im_height = 32
border = 5

In [6]:
X = np.zeros((len(ids), im_height, im_width, 1), dtype=np.float32)
y = np.zeros((len(ids), im_height, im_width, 1), dtype=np.float32)

In [10]:

for n, id_ in tqdm_notebook(enumerate(ids), total=len(ids)):
    # Load images
    img = load_img("data/images/"+id_, grayscale=True)
    x_img = img_to_array(img)
    x_img = resize(x_img, (im_width, im_height, 1), mode = 'constant', preserve_range = True)
    # Load masks
    mask = img_to_array(load_img("data/masks/"+ id_[:-4] + '.png', grayscale=True))
    mask = resize(mask, (im_width, im_height, 1), mode = 'constant', preserve_range = True)
    # Save images
    X[n] = x_img/255.0
    y[n] = mask/255.0
    
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.1, random_state=42)

In [11]:
def conv2d_block(input_tensor, n_filters, kernel_size = 3, batchnorm = True):
    """Function to add 2 convolutional layers with the parameters passed to it"""
    # first layer
    x = Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size),\
              kernel_initializer = 'he_normal', padding = 'same')(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # second layer
    x = Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size),\
              kernel_initializer = 'he_normal', padding = 'same')(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    return x

In [12]:
def get_unet(input_img, n_filters = 16, dropout = 0.1, batchnorm = True):
    """Function to define the UNET Model"""
    # Contracting Path
    c1 = conv2d_block(input_img, n_filters * 1, kernel_size = 3, batchnorm = batchnorm)
    p1 = MaxPooling2D((2, 2))(c1)
    p1 = Dropout(dropout)(p1)
    
    c2 = conv2d_block(p1, n_filters * 2, kernel_size = 3, batchnorm = batchnorm)
    p2 = MaxPooling2D((2, 2))(c2)
    p2 = Dropout(dropout)(p2)
    
    c3 = conv2d_block(p2, n_filters * 4, kernel_size = 3, batchnorm = batchnorm)
    p3 = MaxPooling2D((2, 2))(c3)
    p3 = Dropout(dropout)(p3)
    
    c4 = conv2d_block(p3, n_filters * 8, kernel_size = 3, batchnorm = batchnorm)
    p4 = MaxPooling2D((2, 2))(c4)
    p4 = Dropout(dropout)(p4)
    
    c5 = conv2d_block(p4, n_filters = n_filters * 16, kernel_size = 3, batchnorm = batchnorm)
    
    # Expansive Path
    u6 = Conv2DTranspose(n_filters * 8, (3, 3), strides = (2, 2), padding = 'same')(c5)
    u6 = concatenate([u6, c4])
    u6 = Dropout(dropout)(u6)
    c6 = conv2d_block(u6, n_filters * 8, kernel_size = 3, batchnorm = batchnorm)
    
    u7 = Conv2DTranspose(n_filters * 4, (3, 3), strides = (2, 2), padding = 'same')(c6)
    u7 = concatenate([u7, c3])
    u7 = Dropout(dropout)(u7)
    c7 = conv2d_block(u7, n_filters * 4, kernel_size = 3, batchnorm = batchnorm)
    
    u8 = Conv2DTranspose(n_filters * 2, (3, 3), strides = (2, 2), padding = 'same')(c7)
    u8 = concatenate([u8, c2])
    u8 = Dropout(dropout)(u8)
    c8 = conv2d_block(u8, n_filters * 2, kernel_size = 3, batchnorm = batchnorm)
    
    u9 = Conv2DTranspose(n_filters * 1, (3, 3), strides = (2, 2), padding = 'same')(c8)
    u9 = concatenate([u9, c1])
    u9 = Dropout(dropout)(u9)
    c9 = conv2d_block(u9, n_filters * 1, kernel_size = 3, batchnorm = batchnorm)
    
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)
    model = Model(inputs=[input_img], outputs=[outputs])
    return model

In [13]:
input_img = Input((im_height, im_width, 1), name='img')
model = get_unet(input_img, n_filters=16, dropout=0.05, batchnorm=True)
model.compile(optimizer=Adam(), loss="binary_crossentropy", metrics=["accuracy"])

In [14]:
model.summary()


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
img (InputLayer)                (None, 32, 32, 1)    0                                            
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 32, 32, 16)   160         img[0][0]                        
__________________________________________________________________________________________________
batch_normalization_2 (BatchNor (None, 32, 32, 16)   64          conv2d_2[0][0]                   
__________________________________________________________________________________________________
activation_2 (Activation)       (None, 32, 32, 16)   0           batch_normalization_2[0][0]      
____________________________________________________________________________________________

dropout_7 (Dropout)             (None, 16, 16, 64)   0           concatenate_3[0][0]              
__________________________________________________________________________________________________
conv2d_16 (Conv2D)              (None, 16, 16, 32)   18464       dropout_7[0][0]                  
__________________________________________________________________________________________________
batch_normalization_16 (BatchNo (None, 16, 16, 32)   128         conv2d_16[0][0]                  
__________________________________________________________________________________________________
activation_16 (Activation)      (None, 16, 16, 32)   0           batch_normalization_16[0][0]     
__________________________________________________________________________________________________
conv2d_transpose_4 (Conv2DTrans (None, 32, 32, 16)   4624        activation_16[0][0]              
__________________________________________________________________________________________________
concatenat

In [16]:
callbacks = [
    EarlyStopping(patience=10, verbose=1),
    ReduceLROnPlateau(factor=0.1, patience=5, min_lr=0.00001, verbose=1),
    ModelCheckpoint('model-tgs-salt.h5', verbose=1, save_best_only=True, save_weights_only=True)
]

In [17]:
results = model.fit(X_train, y_train, batch_size=32, epochs=5, callbacks=callbacks,\
                    validation_data=(X_valid, y_valid))

Train on 4500 samples, validate on 500 samples
Epoch 1/5
4500/4500 [==============================] - ETA: 5:16 - loss: 0.8059 - accuracy: 0.01 - ETA: 2:50 - loss: 0.7674 - accuracy: 0.02 - ETA: 2:01 - loss: 0.7313 - accuracy: 0.02 - ETA: 1:36 - loss: 0.7104 - accuracy: 0.02 - ETA: 1:22 - loss: 0.7027 - accuracy: 0.02 - ETA: 1:12 - loss: 0.6938 - accuracy: 0.02 - ETA: 1:05 - loss: 0.6924 - accuracy: 0.02 - ETA: 1:00 - loss: 0.6888 - accuracy: 0.02 - ETA: 56s - loss: 0.6843 - accuracy: 0.0219 - ETA: 52s - loss: 0.6837 - accuracy: 0.020 - ETA: 50s - loss: 0.6846 - accuracy: 0.019 - ETA: 47s - loss: 0.6792 - accuracy: 0.018 - ETA: 45s - loss: 0.6765 - accuracy: 0.018 - ETA: 43s - loss: 0.6734 - accuracy: 0.018 - ETA: 42s - loss: 0.6699 - accuracy: 0.017 - ETA: 42s - loss: 0.6668 - accuracy: 0.019 - ETA: 42s - loss: 0.6660 - accuracy: 0.019 - ETA: 42s - loss: 0.6624 - accuracy: 0.020 - ETA: 42s - loss: 0.6597 - accuracy: 0.020 - ETA: 42s - loss: 0.6582 - accuracy: 0.021 - ETA: 42s - loss: 

4500/4500 [==============================] - ETA: 34s - loss: 0.6231 - accuracy: 0.031 - ETA: 35s - loss: 0.6245 - accuracy: 0.029 - ETA: 36s - loss: 0.6236 - accuracy: 0.022 - ETA: 38s - loss: 0.6211 - accuracy: 0.019 - ETA: 39s - loss: 0.6194 - accuracy: 0.017 - ETA: 40s - loss: 0.6196 - accuracy: 0.021 - ETA: 41s - loss: 0.6238 - accuracy: 0.019 - ETA: 42s - loss: 0.6241 - accuracy: 0.018 - ETA: 41s - loss: 0.6244 - accuracy: 0.018 - ETA: 41s - loss: 0.6255 - accuracy: 0.019 - ETA: 40s - loss: 0.6242 - accuracy: 0.018 - ETA: 40s - loss: 0.6232 - accuracy: 0.017 - ETA: 40s - loss: 0.6249 - accuracy: 0.018 - ETA: 39s - loss: 0.6227 - accuracy: 0.019 - ETA: 39s - loss: 0.6216 - accuracy: 0.019 - ETA: 39s - loss: 0.6220 - accuracy: 0.019 - ETA: 39s - loss: 0.6224 - accuracy: 0.017 - ETA: 39s - loss: 0.6226 - accuracy: 0.019 - ETA: 39s - loss: 0.6231 - accuracy: 0.020 - ETA: 39s - loss: 0.6236 - accuracy: 0.019 - ETA: 39s - loss: 0.6234 - accuracy: 0.020 - ETA: 39s - loss: 0.6229 - accur

KeyboardInterrupt: 